# 📚 Imports

In [1]:
import numpy as np
import pandas as pd
from cleanup.cleanup_siret_functions import clean_numeros

# 📁 Data

In [2]:
df = pd.read_parquet('data/all_communities_data.parquet')

In [3]:
df.head()

,nom,siren,type,cog,cog_3digits,code_departement,code_departement_3digits,code_region,population,epci,...,id_datagouv,merge,ptf,is_active,raison_sociale,raison_sociale_prenom,naf8,code_ju,tranche_effectif,effectifs_sup_50
0,Guadeloupe,239710015,REG,01,None,None,None,None,396153,None,...,None,ptf,KaruData,True,CONSEIL REGIONAL DE LA GUADELOUPE,None,8411Z,7230,1000.0,True
1,Martinique,200055507,CTU,02,None,None,None,None,377711,None,...,None,None,None,True,COLLECTIVITE TERRITORIALE DE MARTINIQUE,None,8411Z,7229,2000.0,True
2,Guyane,200052678,CTU,03,None,None,None,None,271124,None,...,None,None,None,True,COLLECTIVITE TERRITORIALE DE GUYANE,None,8411Z,7229,2000.0,True
3,La Réunion,239740012,REG,04,None,None,None,None,863063,None,...,None,ptf,La Réunion,True,REGION REUNION,None,8411Z,7230,2000.0,True
4,Île-de-France,237500079,REG,11,None,None,None,None,12291279,None,...,534fffa8a3a7292c64a780c8,ptf+datagouv,"Ile-de-France, DataGouv",True,REGION ILE DE FRANCE,None,8411Z,7230,NaN,False


In [4]:
df.shape

(36351, 22)

In [5]:
df.dtypes

nom                          object
siren                        object
type                         object
cog                          object
cog_3digits                  object
code_departement             object
code_departement_3digits     object
code_region                  object
population                    int64
epci                         object
url_ptf                      object
url_datagouv                 object
id_datagouv                  object
merge                        object
ptf                          object
is_active                      bool
raison_sociale               object
raison_sociale_prenom        object
naf8                         object
code_ju                       int64
tranche_effectif            float64
effectifs_sup_50               bool
dtype: object

In [6]:
df.isna().sum()/len(df)

nom                         0.000000
siren                       0.000000
type                        0.000000
cog                         0.034855
cog_3digits                 0.997304
code_departement            0.003164
code_departement_3digits    0.003164
code_region                 0.000468
population                  0.000000
epci                        0.039284
url_ptf                     0.989134
url_datagouv                0.983274
id_datagouv                 0.984099
merge                       0.976809
ptf                         0.977607
is_active                   0.000000
raison_sociale              0.000000
raison_sociale_prenom       1.000000
naf8                        0.000000
code_ju                     0.000000
tranche_effectif            0.025969
effectifs_sup_50            0.000000
dtype: float64

# 🧹Nettoyage

## 👥 Doublons

In [7]:
df[df.duplicated(keep=False)]

,nom,siren,type,cog,cog_3digits,code_departement,code_departement_3digits,code_region,population,epci,...,id_datagouv,merge,ptf,is_active,raison_sociale,raison_sociale_prenom,naf8,code_ju,tranche_effectif,effectifs_sup_50
24999,Aincille,216400119,COM,64011,None,64,064,75,118,200067106,...,614359a8d6f6b32470dfe28b,datagouv,DataGouv,True,COMMUNE D AINCILLE,None,8411Z,7210,3.0,False
25000,Aincille,216400119,COM,64011,None,64,064,75,118,200067106,...,614359a8d6f6b32470dfe28b,datagouv,DataGouv,True,COMMUNE D AINCILLE,None,8411Z,7210,3.0,False


On supprime la ligne en doublon

In [8]:
df = df.drop_duplicates()

In [9]:
df.shape

(36350, 22)

## Unicité SIREN

In [10]:
df['siren'].nunique()

36344

In [11]:
df['siren'].duplicated().sum()

np.int64(6)

In [12]:
df[df['siren'].duplicated(keep=False)].sort_values(by='siren')

,nom,siren,type,cog,cog_3digits,code_departement,code_departement_3digits,code_region,population,epci,...,id_datagouv,merge,ptf,is_active,raison_sociale,raison_sociale_prenom,naf8,code_ju,tranche_effectif,effectifs_sup_50
85,Métropole de Lyon,200046977,DEP,691,691,None,None,84,1402326,None,...,534fff7aa3a7292c64a77e0c,ptf+datagouv,"Métropole de Lyon, DataGouv",True,METROPOLE DE LYON,None,8411Z,7229,5000.0,True
35347,Métropole de Lyon,200046977,MET,None,None,69,069,84,1402326,None,...,534fff7aa3a7292c64a77e0c,ptf+datagouv,"Métropole de Lyon, DataGouv",True,METROPOLE DE LYON,None,8411Z,7229,5000.0,True
35817,Métropole Européenne de Lille,200093201,MET,None,None,59,059,32,24566,None,...,None,None,None,True,METROPOLE EUROPEENNE DE LILLE,None,8411Z,7344,2000.0,True
35818,Métropole Européenne de Lille,200093201,MET,None,None,59,059,32,1157126,None,...,None,None,None,True,METROPOLE EUROPEENNE DE LILLE,None,8411Z,7344,2000.0,True
35819,Métropole Européenne de Lille,200093201,MET,None,None,59,059,32,1181692,None,...,None,None,None,True,METROPOLE EUROPEENNE DE LILLE,None,8411Z,7344,2000.0,True
35821,CA Agglomération d'Agen,200096956,CA,None,None,47,047,75,5372,None,...,None,None,None,True,AGGLOMERATION D'AGEN,None,8411Z,7348,1000.0,True
35822,CA Agglomération d'Agen,200096956,CA,None,None,47,047,75,99115,None,...,None,None,None,True,AGGLOMERATION D'AGEN,None,8411Z,7348,1000.0,True
91,Paris,217500016,DEP,75,075,None,None,11,2204773,None,...,534fff89a3a7292c64a77eb7,ptf+datagouv,"Paris, DataGouv",True,VILLE DE PARIS,None,8411Z,7229,NaN,False
29388,Paris,217500016,COM,75056,None,75,075,11,2204773,200054781,...,534fff89a3a7292c64a77eb7,ptf+datagouv,"Paris, DataGouv",True,VILLE DE PARIS,None,8411Z,7229,NaN,False
36001,CA du Pays de l'Or,243400470,CA,None,None,34,034,76,45153,None,...,637f49997fec449b313a8811,datagouv,DataGouv,True,COMM D'AGGLOMERATION DU PAYS DE L'OR,None,8411Z,7348,500.0,True


⚠️ Sujet des doublons de SIREN à gérer par la pipeline...

In [13]:
df.siren.isna().sum()

np.int64(0)

Pas de valeurs nulles

## Drop colonnes non utiles pour le Web (mais à garder si besoin côté Data Eng)

In [14]:
df = df.drop(columns=['cog_3digits', 'code_departement_3digits', 'url_ptf', 'url_datagouv','id_datagouv', 'merge', 'ptf'])

In [15]:
df.head()

,nom,siren,type,cog,code_departement,code_region,population,epci,is_active,raison_sociale,raison_sociale_prenom,naf8,code_ju,tranche_effectif,effectifs_sup_50
0,Guadeloupe,239710015,REG,01,None,None,396153,None,True,CONSEIL REGIONAL DE LA GUADELOUPE,None,8411Z,7230,1000.0,True
1,Martinique,200055507,CTU,02,None,None,377711,None,True,COLLECTIVITE TERRITORIALE DE MARTINIQUE,None,8411Z,7229,2000.0,True
2,Guyane,200052678,CTU,03,None,None,271124,None,True,COLLECTIVITE TERRITORIALE DE GUYANE,None,8411Z,7229,2000.0,True
3,La Réunion,239740012,REG,04,None,None,863063,None,True,REGION REUNION,None,8411Z,7230,2000.0,True
4,Île-de-France,237500079,REG,11,None,None,12291279,None,True,REGION ILE DE FRANCE,None,8411Z,7230,NaN,False


## Formattage

In [16]:
df.dtypes

nom                       object
siren                     object
type                      object
cog                       object
code_departement          object
code_region               object
population                 int64
epci                      object
is_active                   bool
raison_sociale            object
raison_sociale_prenom     object
naf8                      object
code_ju                    int64
tranche_effectif         float64
effectifs_sup_50            bool
dtype: object

### Siren

Utilisation de la fonction de formattage des SIREN faite par Chloé

In [17]:
df.loc[:,'siren'] = df['siren'].apply(clean_numeros)

In [18]:
df.head()

,nom,siren,type,cog,code_departement,code_region,population,epci,is_active,raison_sociale,raison_sociale_prenom,naf8,code_ju,tranche_effectif,effectifs_sup_50
0,Guadeloupe,239710015,REG,01,None,None,396153,None,True,CONSEIL REGIONAL DE LA GUADELOUPE,None,8411Z,7230,1000.0,True
1,Martinique,200055507,CTU,02,None,None,377711,None,True,COLLECTIVITE TERRITORIALE DE MARTINIQUE,None,8411Z,7229,2000.0,True
2,Guyane,200052678,CTU,03,None,None,271124,None,True,COLLECTIVITE TERRITORIALE DE GUYANE,None,8411Z,7229,2000.0,True
3,La Réunion,239740012,REG,04,None,None,863063,None,True,REGION REUNION,None,8411Z,7230,2000.0,True
4,Île-de-France,237500079,REG,11,None,None,12291279,None,True,REGION ILE DE FRANCE,None,8411Z,7230,NaN,False


### COG

Types de collectivités où le cog est null :

In [19]:
df[df['cog'].isna()].type.value_counts()

type
CC     989
CA     229
MET     24
CU      14
EPT     11
Name: count, dtype: int64

Tout est ok

### Code département

In [20]:
df[df['code_departement'].isna()].type.value_counts()

type
DEP    98
REG    14
CTU     3
Name: count, dtype: int64

### Code région

In [21]:
df[df['code_region'].isna()].type.value_counts()

type
REG    14
CTU     3
Name: count, dtype: int64

### EPCI

### Formattage cog, code_region, code_departement

In [ ]:
def upper_strip(code):
        if pd.isna(code) or str(code).strip().lower() in {"non renseigné", "non", "nan", "none", ""}:
           return np.nan
        else :
            return str(code).strip().upper()

In [23]:
df['code_region']=df['code_region'].map(upper_strip)
df['code_departement']=df['code_departement'].map(upper_strip)
df['cog']=df['cog'].map(upper_strip)

In [24]:
df.head()

,nom,siren,type,cog,code_departement,code_region,population,epci,is_active,raison_sociale,raison_sociale_prenom,naf8,code_ju,tranche_effectif,effectifs_sup_50
0,Guadeloupe,239710015,REG,01,NaN,NaN,396153,None,True,CONSEIL REGIONAL DE LA GUADELOUPE,None,8411Z,7230,1000.0,True
1,Martinique,200055507,CTU,02,NaN,NaN,377711,None,True,COLLECTIVITE TERRITORIALE DE MARTINIQUE,None,8411Z,7229,2000.0,True
2,Guyane,200052678,CTU,03,NaN,NaN,271124,None,True,COLLECTIVITE TERRITORIALE DE GUYANE,None,8411Z,7229,2000.0,True
3,La Réunion,239740012,REG,04,NaN,NaN,863063,None,True,REGION REUNION,None,8411Z,7230,2000.0,True
4,Île-de-France,237500079,REG,11,NaN,NaN,12291279,None,True,REGION ILE DE FRANCE,None,8411Z,7230,NaN,False


# CONCLUSIONS - A FAIRE COTE DATA ENG

#### 1. Gérer les doublons de SIREN (le plus important)
#### 2. Nettoyage
Juste du reformattage en utilisant clean_numeros et une petite fonction strip_upper
#### 3. Clarifier `naf8` et `code_ju` (nouvelles données)
#### 4. Traitement EPCI à affiner
avec les data analystes en fonction de l'exploitation faite côté Front
#### 5. Données à ajouter
    - Latitude et longitude
    - Superficie
    - Densité
    - Obligation publication (bool) : se calcule à partir de population et effectifs_sup_50
    - Type_elu (maire, président région, etc en fonction de type)
    - Nom_elu

#### 6. Colonnes à renommer